In [ ]:
#My VS Code has issues connecting to the venv.
# I find that first connecting to one of my conda environments
# and then switching to the venv works.
# I run this cell on conda environment first then switch to venv
# and run it there and seems to be all good.
print("Hello")

In [ ]:
# This cell allows hot reloading of modules.
# It is useful for development, so you don't have to restart the kernel.
# Some changes such as adding new files or changing the structure of the code
# may not be picked up and will require a kernel restart.
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import requests
import time
from dotenv import load_dotenv, find_dotenv
import logging

# Set up logging include logger name
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

from kapipy.gis import GIS
from kapipy.gis import has_geopandas, has_arcgis, has_arcpy

if has_arcgis:
    import pandas as pd
    from arcgis.features import GeoAccessor, GeoSeriesAccessor

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
api_key = os.getenv('LRIS_API_KEY')

output_folder = r"c:\temp\data"

#layers
lcdb_layer_id = "104400" #Land Cover Database - 511,104 polygons
nzlri_layer_id = "48066" #NZLRI Soils - 63,516 polygons
mpi_slmacc_layer_id = "48574" # MPI SLMACC Auger and Tacit Soil Observations, 672 points

#tables - there are no tables in the LRIS portal!

print(f'{has_arcgis=}')
print(f'{has_arcpy=}')
print(f'{has_geopandas=}')

assert(has_geopandas == False)
assert(has_arcgis == True)

## Server object  
Get a reference to the LRIS server  

In [ ]:
lris = GIS(name="lris", api_key=api_key)
print(lris)

## Layer item  
Run some tests on a layer item  

In [ ]:
itm = lris.content.get(lcdb_layer_id)
print(itm)

In [ ]:
print(f'{itm.supports_changesets=}')
print(f'{itm.data.fields=}')
print(f'{itm.data.crs=}')
print(f'{itm.data.geometry_type=}')

In [ ]:
# read shapefile into an SDF
matamata_sdf = pd.DataFrame.spatial.from_featureclass("../examples/matamata_piako.shp")
print(matamata_sdf.spatial.geometry_type)
matamata_sdf.head()

In [ ]:
#Query using WFS
res = itm.query(count=100, out_sr=2193, bbox=matamata_sdf)
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()

In [ ]:
job = itm.export("geodatabase", out_sr=2193, extent=matamata_sdf,)

In [ ]:
print(job)

In [ ]:
dl = job.download(folder=output_folder)

In [ ]:
print(f'{dl.folder=}')
print(f'{dl.filename=}')
print(f'{dl.file_path=}')
print(f'{dl.file_size_bytes=}')
print(f'{dl.download_url=}')
print(f'{dl.final_url=}')
print(f'{dl.job_id=}')
print(f'{dl.completed_at=}')
print(f'{dl.checksum=}')

## Test multidownload samples  
Test downloading multiple jobs using the ContentManager download method.    

In [ ]:
itm1 = lris.content.get(lcdb_layer_id)
itm2 = lris.content.get(nzlri_layer_id)
print(itm1)
print(itm2)

In [ ]:
job1 = itm1.export("geodatabase", out_sr=2193, extent=matamata_sdf,)
job2 = itm2.export("geodatabase", out_sr=2193, extent=matamata_sdf,)

results = lris.content.download(jobs=[job1, job2], folder=output_folder)

In [ ]:
for result in results:
    print(result.download_file_path)